# Pandas MultiIndexing
This document will go into using dataframes with multi-indexes to accomplish common tasks. There are many ways to do each of the things described in this doc, but the examples given are the best way that I have found to do them.

In [75]:
import numpy as np
import pandas as pd
import itertools

## 1 Regular Indexes
Indexes are straightforward, so they will be used as a starting point for understindg multiindexes.

### 1.1 Construct Regular Indexes

In [44]:
# construct from a list or iterable
ind_cols = pd.Index(['a','b', 'c'])
print('List-based Index:', ind_cols)
ind_rows = pd.Index(range(4))
print('Range-based Index:', ind_rows)

List-based Index: Index(['a', 'b', 'c'], dtype='object')
Range-based Index: Int64Index([0, 1, 2, 3], dtype='int64')


### 1.2 Use Indexes to Build a DataFrame

Next, we'll build a dataframe that uses these indices for index and column dimensions. These are the two default dimensions, and are sufficient for non-complex tabular data.

In [45]:
df = pd.DataFrame(index=ind_rows,columns=ind_cols)
print(df)

     a    b    c
0  NaN  NaN  NaN
1  NaN  NaN  NaN
2  NaN  NaN  NaN
3  NaN  NaN  NaN


In [46]:
# and this time with numbers
df = pd.DataFrame(np.random.uniform(0,1,size=(4,3)),index=ind_rows,columns=ind_cols)
print(df)

          a         b         c
0  0.079523  0.079244  0.800245
1  0.707402  0.314167  0.411310
2  0.994567  0.863539  0.754527
3  0.309907  0.994755  0.478533


### 1.3 Column Indexing
Column selection can be done using df[col_name] and row selection using a logical index like df[logical_index] where logical_index is the result of an operation like df[col_name] > 4.0. 

In [47]:
df['b']

0    0.079244
1    0.314167
2    0.863539
3    0.994755
Name: b, dtype: float64

### 1.4 Logical Indexing
Logical comparisons can be done to index into a dataframe. Compound logic can be done with the numpy functions np.logical_not, np.logical_and, np.logical_or, etc. Note that this method of indexing is quite slow because it must compare every element and then use that to index the dataframe.

In [48]:
df[df['a'] > 0.2]

,a,b,c
1,0.707402,0.314167,0.411310
2,0.994567,0.863539,0.754527
3,0.309907,0.994755,0.478533


In [49]:
df[np.logical_and(df['a'] > 0.1, df['b'] > 0.1)]

,a,b,c
1,0.707402,0.314167,0.411310
2,0.994567,0.863539,0.754527
3,0.309907,0.994755,0.478533


### 1.5 .loc[] Indexing
The best or most common way of indexing uses the .loc method of a dataframe, but it can also be the most challenging. Dimensions (like index and columns) are always separated by commas in the loc[] brackets.

In [54]:
df.loc[1,'a']

0.70740160428090126

Slicing can also used on each dimension.

In [57]:
df.loc[1:,'a':'b']

,a,b
1,0.707402,0.314167
2,0.994567,0.863539
3,0.309907,0.994755


When accessing dimensions that use Simple Indexes, providing a tuple will allow one to explicitly select multiple columns. Be careful, because this has a different meaning with multi-indexes.

In [59]:
df.loc[1:,('a','c')]

,a,c
1,0.707402,0.411310
2,0.994567,0.754527
3,0.309907,0.478533


## 2 The MultiIndex
There are a couple different ways to construct and use a multi-index. First thing to note is that a pandas MultiIndex object can be used instead of a regular Index on any dimension (like index, columns). First, there are a few ways to construct them.

### 2.1 Construct a MultiIndex

In [77]:
nodes = range(4)
attr = ('in', 'out')
indlist = list(itertools.product(nodes,attr))
mi = pd.MultiIndex.from_tuples(indlist, names=['number','direction'])
print(mi)

MultiIndex(levels=[[0, 1, 2, 3, 4], ['in', 'out']],
           labels=[[0, 0, 1, 1, 2, 2, 3, 3, 4, 4], [0, 1, 0, 1, 0, 1, 0, 1, 0, 1]],
           names=['number', 'direction'])


or, equivalently:

In [132]:
nodes = range(4)
attr = ('in', 'out')
mi = pd.MultiIndex.from_product([nodes,attr], names=['number','direction'])
print(mi)

MultiIndex(levels=[[0, 1, 2, 3], ['in', 'out']],
           labels=[[0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 0, 1, 0, 1, 0, 1]],
           names=['number', 'direction'])


A MultiIndex itself is an efficient structure that is a little hard to view with a simple print. One of the easiest ways to observe it might be in list form. You'll have to imagine the heararchy part this way. It's also convenient to see the heirarchy later in the DataFrame view.

In [79]:
print(list(mi))

[(0, 'in'), (0, 'out'), (1, 'in'), (1, 'out'), (2, 'in'), (2, 'out'), (3, 'in'), (3, 'out'), (4, 'in'), (4, 'out')]


You can access these items directly to get all possible labels at each level.

In [71]:
mi.levels[1]

Index(['in', 'out'], dtype='object')

And you can also use the method "get_level_values" to acess different pseudo-columns of the index.

In [123]:
print(mi.get_level_values(1))
print(mi.get_level_values('direction'))

Index(['in', 'out', 'in', 'out', 'in', 'out', 'in', 'out'], dtype='object', name='direction')
Index(['in', 'out', 'in', 'out', 'in', 'out', 'in', 'out'], dtype='object', name='direction')


You can also reorder the index levels if needed.

In [133]:
mi = mi.reorder_levels((1,0))
print(mi)
mi = mi.reorder_levels((1,0))
print(mi)

MultiIndex(levels=[['in', 'out'], [0, 1, 2, 3]],
           labels=[[0, 1, 0, 1, 0, 1, 0, 1], [0, 0, 1, 1, 2, 2, 3, 3]],
           names=['direction', 'number'])
MultiIndex(levels=[[0, 1, 2, 3], ['in', 'out']],
           labels=[[0, 0, 1, 1, 2, 2, 3, 3], [0, 1, 0, 1, 0, 1, 0, 1]],
           names=['number', 'direction'])


### 2.2 Construct a DataFrame Using a MultiIndex
Now the MultiIndex will be used to construct a dataframe. Without data values it will look like this:

In [89]:
cols = ['attr_a','attr_b','attr_c']
df = pd.DataFrame(index=mi,columns=cols)
print(df)

                 attr_a attr_b attr_c
number direction                     
0      in           NaN    NaN    NaN
       out          NaN    NaN    NaN
1      in           NaN    NaN    NaN
       out          NaN    NaN    NaN
2      in           NaN    NaN    NaN
       out          NaN    NaN    NaN
3      in           NaN    NaN    NaN
       out          NaN    NaN    NaN


and with data values, like this:

In [90]:
cols = ['attr_a','attr_b','attr_c']
dat = np.random.uniform(0,1,(len(mi),len(cols)))
df = pd.DataFrame(dat,index=mi,columns=cols)
print(df)

                    attr_a    attr_b    attr_c
number direction                              
0      in         0.328582  0.090670  0.287683
       out        0.299169  0.216481  0.337355
1      in         0.648115  0.864053  0.373318
       out        0.053789  0.497447  0.164034
2      in         0.190671  0.796935  0.875535
       out        0.808041  0.903385  0.917301
3      in         0.840225  0.849640  0.894982
       out        0.409888  0.753380  0.278426


Although in this case it has already been done, in some cases you may need to sort the axis dimensions to perform partial indexing.

In [97]:
df.sort_index(axis='index',inplace=True)

### 2.3 .loc[] Indexing With MultiIndex
Just as with simple index dataframes, .loc[] for dataframes with multiindices should separate dimensions by commas. In dimensions that use the MultiIndex, provide a complete tuple to get or assign a specific value.

In [104]:
df.loc[(1,'in'),'attr_a']

0.64811475721524792

And ignore all but the first index level by providng a scalar.

In [106]:
df.loc[1,'attr_a']

direction
in     0.648115
out    0.053789
Name: attr_a, dtype: float64

In this example, you can consider the tuple (0,'in') to be a single element of the MultiIndex. Because of that, you can slice values like the following.

In [94]:
df.loc[(0,'in'):(1,'out'),'attr_a']

number  direction
0       in           0.328582
        out          0.299169
1       in           0.648115
        out          0.053789
Name: attr_a, dtype: float64

### 2.4 .loc[] With Partial Index

You can also provide a partial index leveling in order.

In [95]:
df.loc[(0,),'attr_a']

direction
in     0.328582
out    0.299169
Name: attr_a, dtype: float64

Also include slices in tuples. Because it is in the tuple you need to use the 'slice' function instead of the ':' operator.

In [100]:
df.loc[(slice(0,1),),'attr_a']

number  direction
0       in           0.328582
        out          0.299169
1       in           0.648115
        out          0.053789
Name: attr_a, dtype: float64

Or use another tuple to get specific values at a specific level.

In [99]:
df.loc[((0,3),),'attr_a']

number  direction
0       in           0.328582
        out          0.299169
3       in           0.840225
        out          0.409888
Name: attr_a, dtype: float64

Incomplete indexing can be made more robust by including slicing in specific levels according to the lexical sorting. Note that 'slice(None)' can be used instead of the 'all' slice operator ':'.

In [110]:
df.loc[(slice(None),'out'),'attr_a']

number  direction
0       out          0.299169
1       out          0.053789
2       out          0.808041
3       out          0.409888
Name: attr_a, dtype: float64

## 3 Looping Through DataFrame With MultiIndex Dimension
Looping is one of the most simple things you may want to do with your MultiIndex and DataFrame. You can always get your multiindex back out of the dataframe directly. For example we will switch to a thre-dimensional indexing scheme.

In [171]:
cols = ['skill', 'experience']
mi = pd.MultiIndex.from_product([range(3),('in','out'),('a','b')])
data = np.random.uniform(0,1,size=(12,len(cols)))
df = pd.DataFrame(data,index=mi,columns=cols)
print(df)

            skill  experience
0 in  a  0.430797    0.541542
      b  0.155757    0.154997
  out a  0.233468    0.042139
      b  0.522801    0.938821
1 in  a  0.657610    0.755681
      b  0.722555    0.794694
  out a  0.447087    0.873459
      b  0.573240    0.068749
2 in  a  0.908337    0.422624
      b  0.289635    0.824910
  out a  0.553447    0.532436
      b  0.997006    0.166137


In [172]:
mi.where()

AttributeError: 'MultiIndex' object has no attribute 'where'

In [170]:
for i in df.index.get_level_values(0):
    print(df.loc[(i,),:].index.get_level_values(0))
    for j in df.loc[(i,),:].index.get_level_values(0):
        print(df.loc[(i,j,slice(None)),:].index.get_level_values(0))
        for k in df.loc[(i,j,slice(None)),:].index.get_level_values(0):
            print(str((i,j,k)), df.loc[(i,j,k),:])

Index(['in', 'in', 'out', 'out'], dtype='object')
Int64Index([0, 0], dtype='int64')


KeyError: (0, 'in', 0)

In [160]:
ind = df.loc[(2,),'attr_a'].index
for i in ind:
    print(i)

in
out
